In [ ]:
import numpy as np
import cv2
from copy import deepcopy
from PIL import Image
import pytesseract as tess

def preprocess(img):
    cv2.imshow("Input",img)
    imgBlurred = cv2.GaussianBlur(img, (5,5), 0)
    gray = cv2.cvtColor(imgBlurred, cv2.COLOR_BGR2GRAY)

    sobelx = cv2.Sobel(gray,cv2.CV_8U,1,0,ksize=3)
    #cv2.imshow("Sobel",sobelx)
    #cv2.waitKey(0)
    ret2,threshold_img = cv2.threshold(sobelx,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    #cv2.imshow("Threshold",threshold_img)
    #cv2.waitKey(0)
    return threshold_img

def cleanPlate(plate):
    print ("CLEANING PLATE. . .")
    gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
    #kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    #thresh= cv2.dilate(gray, kernel, iterations=1)

    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    im1,contours,hierarchy = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    if contours:
        areas = [cv2.contourArea(c) for c in contours]
        max_index = np.argmax(areas)

        max_cnt = contours[max_index]
        max_cntArea = areas[max_index]
        x,y,w,h = cv2.boundingRect(max_cnt)

        if not ratioCheck(max_cntArea,w,h):
            return plate,None

        cleaned_final = thresh[y:y+h, x:x+w]
        #cv2.imshow("Function Test",cleaned_final)
        return cleaned_final,[x,y,w,h]

    else:
        return plate,None


def extract_contours(threshold_img):
    element = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(17, 3))
    morph_img_threshold = threshold_img.copy()
    cv2.morphologyEx(src=threshold_img, op=cv2.MORPH_CLOSE, kernel=element, dst=morph_img_threshold)
    cv2.imshow("Morphed",morph_img_threshold)
    cv2.waitKey(0)

    im2,contours, hierarchy= cv2.findContours(morph_img_threshold,mode=cv2.RETR_EXTERNAL,method=cv2.CHAIN_APPROX_NONE)
    return contours


def ratioCheck(area, width, height):
    ratio = float(width) / float(height)
    if ratio < 1:
        ratio = 1 / ratio

    aspect = 4.7272
    min = 15*aspect*15  # minimum area
    max = 125*aspect*125  # maximum area

    rmin = 3
    rmax = 6

    if (area < min or area > max) or (ratio < rmin or ratio > rmax):
        return False
    return True

def isMaxWhite(plate):
    avg = np.mean(plate)
    if(avg>=115):
        return True
    else:
        return False

def validateRotationAndRatio(rect):
    (x, y), (width, height), rect_angle = rect

    if(width>height):
        angle = -rect_angle
    else:
        angle = 90 + rect_angle

    if angle>15:
        return False

    if height == 0 or width == 0:
        return False

    area = height*width
    if not ratioCheck(area,width,height):
        return False
    else:
        return True



def cleanAndRead(img,contours):
    #count=0
    for i,cnt in enumerate(contours):
        min_rect = cv2.minAreaRect(cnt)

        if validateRotationAndRatio(min_rect):

            x,y,w,h = cv2.boundingRect(cnt)
            plate_img = img[y:y+h,x:x+w]


            
                #count+=1
            clean_plate, rect = cleanPlate(plate_img)

            if rect:
                x1,y1,w1,h1 = rect
                x,y,w,h = x+x1,y+y1,w1,h1
                cv2.imshow("Cleaned Plate",clean_plate)
                cv2.waitKey(0)
                plate_im = Image.fromarray(clean_plate)
                text = tess.image_to_string(plate_im, lang='eng')
                print ("Detected Text : ",text)
                img = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
                cv2.imshow("Detected Plate",img)
                cv2.waitKey(0)
                cv2.destroyAllWindows()

    #print "No. of final cont : " , count



if __name__ == '__main__':
    #print("DETECTING PLATE . . .")

    #img = cv2.imread("testData/Final.JPG")
    img = cv2.imread("test.jpeg")

    threshold_img = preprocess(img)
    contours= extract_contours(threshold_img)
    cv2.destroyWindow("Morphed")
    #if len(contours)!=0:
        #print len(contours) #Test
        # cv2.drawContours(img, contours, -1, (0,255,0), 1)
        # cv2.imshow("Contours",img)
        # cv2.waitKey(0)


    cleanAndRead(img,contours)
cv2.destroyAllWindows()

In [1]:
import cv2

In [5]:
cleanAndRead(img,contours)

In [6]:
contours

[array([[[791, 446]],
 
        [[791, 447]],
 
        [[792, 448]],
 
        [[792, 449]],
 
        [[793, 449]],
 
        [[794, 449]],
 
        [[795, 449]],
 
        [[796, 449]],
 
        [[797, 449]],
 
        [[798, 449]],
 
        [[799, 449]],
 
        [[799, 448]],
 
        [[799, 447]],
 
        [[799, 446]],
 
        [[798, 446]],
 
        [[797, 446]],
 
        [[796, 446]],
 
        [[795, 446]],
 
        [[794, 446]],
 
        [[793, 446]],
 
        [[792, 446]]], dtype=int32), array([[[354, 444]],
 
        [[354, 445]],
 
        [[354, 446]],
 
        [[354, 447]],
 
        [[354, 448]],
 
        [[354, 449]],
 
        [[355, 449]],
 
        [[356, 449]],
 
        [[357, 449]],
 
        [[358, 449]],
 
        [[359, 449]],
 
        [[360, 449]],
 
        [[361, 449]],
 
        [[362, 449]],
 
        [[363, 449]],
 
        [[364, 449]],
 
        [[365, 449]],
 
        [[366, 449]],
 
        [[367, 449]],
 
        [[368, 449]],
 
    

In [ ]:
def cleanAndRead(img,contours):
    #count=0
    for i,cnt in enumerate(contours):
        min_rect = cv2.minAreaRect(cnt)

        if validateRotationAndRatio(min_rect):

            x,y,w,h = cv2.boundingRect(cnt)
            plate_img = img[y:y+h,x:x+w]


            if(isMaxWhite(plate_img)):
                #count+=1
                clean_plate, rect = cleanPlate(plate_img)

                if rect:
                    x1,y1,w1,h1 = rect
                    x,y,w,h = x+x1,y+y1,w1,h1
                    cv2.imshow("Cleaned Plate",clean_plate)
                    cv2.waitKey(0)
                    plate_im = Image.fromarray(clean_plate)
                    text = tess.image_to_string(plate_im, lang='eng')
                    print ("Detected Text : ",text)
                    img = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
                    cv2.imshow("Detected Plate",img)
                    cv2.waitKey(0)


In [2]:
cv2.destroyAllWindows()

In [4]:
import numpy as np
labels=np.load('all_words_labels.npy')
lip_data=np.load('all_words.npy')

In [5]:
import pandas as pd

In [6]:
lab=pd.DataFrame(labels)

In [7]:
lab.to_csv('dataset.csv')

In [29]:
previous=-1
for i,value in enumerate(labels):
    if i%10 ==0:
        previous=(previous+1)%10
    if previous!=value:
        print('False')
    

False


In [27]:
labels[1400]=2

In [24]:
labels=np.append(labels,2)

In [13]:
x=lip_data.reshape(-1,20)

In [22]:
x.T

array([[ 1.61302962,  1.48218093,  1.39822034, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.35375527,  0.3718258 ,  0.42261517, ...,  0.        ,
         0.        ,  0.        ],
       [-0.27812157,  0.01152839,  0.06671332, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.81067945, -0.9690925 , -1.11109535, ...,  0.        ,
         0.        ,  0.        ],
       [-1.75582715, -1.78271177, -1.76217198, ...,  0.        ,
         0.        ,  0.        ],
       [-0.81067945, -1.19628818, -1.11109535, ...,  0.        ,
         0.        ,  0.        ]])

In [26]:
from keras.models import load_model
model=load_model('minor2.h5')

In [45]:
y=x.T.reshape(20,1500,10,1)

In [42]:
y[0]

array([[1.61302962, 1.48218093, 1.39822034, ..., 1.40203631, 1.67515766,
        1.8727403 ],
       [1.49365298, 1.47186779, 1.60462004, ..., 0.        , 0.        ,
        0.        ],
       [1.80020661, 1.52245004, 1.42392307, ..., 1.60408088, 1.68994346,
        1.82825507],
       ...,
       [1.81267042, 1.49222402, 1.81616422, ..., 0.        , 0.        ,
        0.        ],
       [1.88774008, 1.71860302, 1.47134619, ..., 0.        , 0.        ,
        0.        ],
       [1.66682155, 1.67624997, 1.28406203, ..., 0.        , 0.        ,
        0.        ]])

In [37]:
pd.DataFrame(lip_data).to_csv('data.csv')

ValueError: Must pass 2-d input

In [50]:
lip_data[0]

array([[ 1.61302962,  0.35375527, -0.27812157, -0.95284227, -0.0693621 ,
         0.61050796,  1.87905196,  0.56714919, -0.0693621 , -0.14985739,
         0.15318074,  0.93350614,  1.08186833, -0.81067945, -1.75582715,
        -0.81067945,  1.08186833, -0.81067945, -1.75582715, -0.81067945],
       [ 1.48218093,  0.3718258 ,  0.01152839, -0.42667965,  0.13261789,
         0.6171553 ,  1.74297172,  0.73832283,  0.13261789, -0.15547323,
         0.13261789,  0.73832283,  0.92935246, -1.07606953, -1.78271177,
        -0.8410462 ,  1.20055888, -0.9690925 , -1.78271177, -1.19628818],
       [ 1.39822034,  0.42261517,  0.06671332, -0.44534185,  0.06671332,
         0.65561766,  1.66158636,  0.78244523,  0.30495051,  0.08139021,
         0.30495051,  0.92588802,  0.60812226, -1.11109535, -1.76217198,
        -1.11109535,  1.13485431, -1.11109535, -1.76217198, -1.11109535],
       [ 1.70378078,  0.58617932, -0.32463355, -0.83410664, -0.32463355,
         0.58617932,  1.9450976 ,  0.5277123 , -

In [51]:
y[1][0]

array([[0.35375527],
       [0.3718258 ],
       [0.42261517],
       [0.58617932],
       [0.48887405],
       [0.57365214],
       [0.36488715],
       [0.50087724],
       [0.55427544],
       [0.51062977]])

In [1]:
def fun():
    print(2)

In [2]:
x=[]
x.append(fun())

2


In [3]:
x

[None]

In [6]:
import numpy as np
np.array([[[1,2,3],[1,4,7]],[[1,2,3],[1,4,7],[5,8,9]],[[1,2,3]]]).shape

(3,)

In [7]:
sequence_length = np.random.randint(10, 100)
x_train = np.random.random((1000, sequence_length, 5))

In [16]:
x=np.array([1,3,4,65,66,7,7,4,32])

In [17]:
x.shape=(3,None)

TypeError: 'NoneType' object cannot be interpreted as an integer

In [18]:
import keras

C:\Users\deepu\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
keras.